In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
start = '2015-01-01'

In [3]:
overall = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

In [4]:
overall

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [5]:
# we need only symbols:
stocks = overall.Symbol

In [6]:
stocks.head()

0     MMM
1     AOS
2     ABT
3    ABBV
4     ACN
Name: Symbol, dtype: object

In [7]:
stocks = stocks.to_list()

In [8]:
#stocks

In [9]:
# filter out only companies added after a certain start date:
overall = overall[overall['Date added'] >= start]

In [10]:
removed = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[1][['Date','Removed']]

In [11]:
# removed

In [12]:
removed = removed.set_index(removed.Date.Date)

In [13]:
removed.index = pd.to_datetime(removed.index)

In [14]:
# removed

In [15]:
removed = removed[removed.index >= start]

In [16]:
removed

Date Removed                     
                        Date  Ticker             Security
Date                                                     
2023-08-25   August 25, 2023     AAP   Advance Auto Parts
2023-06-20     June 20, 2023    DISH         Dish Network
2023-05-04       May 4, 2023     FRC  First Republic Bank
2023-03-20    March 20, 2023    LUMN   Lumen Technologies
2023-03-15    March 15, 2023    SBNY       Signature Bank
...                      ...     ...                  ...
2015-03-23    March 23, 2015     AVP        Avon Products
2015-03-18    March 18, 2015     CFN           Carefusion
2015-03-12    March 12, 2015    PETM             PetSmart
2015-01-27  January 27, 2015     SWY              Safeway
2015-01-27  January 27, 2015     COV             Covidien

[198 rows x 3 columns]

In [17]:
removed = removed.Removed.dropna()

In [18]:
removed

,Ticker,Security
Date,,
2023-08-25,AAP,Advance Auto Parts
2023-06-20,DISH,Dish Network
2023-05-04,FRC,First Republic Bank
2023-03-20,LUMN,Lumen Technologies
2023-03-15,SBNY,Signature Bank
...,...,...
2015-03-23,AVP,Avon Products
2015-03-18,CFN,Carefusion
2015-03-12,PETM,PetSmart


In [19]:
# extend removed symbols as well, so that they will be covered by our analysis
stocks.extend(removed.Ticker.to_list())

In [20]:
# pulling the stock Close prices
# alltogether 684 symbols, including 187 delisted ones
df = yf.download(stocks, start = start)['Close']

[*********************100%***********************]  684 of 684 completed


81 Failed downloads:
['MON', 'MXIM', 'APC', 'NLSN', 'PBCT', 'SIVB', 'ADS', 'TWTR', 'AVP', 'VIAB', 'ETFC', 'FRC', 'BBBY', 'CXO', 'DWDP', 'ENDP', 'CERN', 'XL', 'WIN', 'HFC', 'LLL', 'KSU', 'ESV', 'FTR', 'RHT', 'VAR', 'DNR', 'CTXS', 'FBHS', 'XLNX', 'STI', 'ALXN', 'WCG', 'INFO', 'CELG', 'NBL', 'TIF', 'LM', 'AGN', 'DTV', 'XEC', 'QEP', 'DRE', 'TSS', 'FLIR', 'BRK.B', 'RTN']: Exception('%ticker%: No timezone found, symbol may be delisted')
['SWY', 'STJ', 'SIAL', 'CAM', 'WYN', 'LVLT', 'BCR', 'BXLT', 'KRFT', 'CVC', 'YHOO', 'RAI', 'GMCR', 'ARG', 'WFM', 'CPGX', 'GGP', 'LLTC', 'MJN', 'CMCSK', 'BF.B', 'TYC', 'JOY', 'SNDK', 'SPLS', 'HSP', 'LO', 'HCBK', 'DPS', 'TWC', 'FDO', 'TE', 'CFN', 'BRCM']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2015-01-01 -> 2023-09-01)')


In [21]:
df.index = pd.to_datetime(df.index)

In [22]:
df

,A,AA,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACE,...,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,40.560001,38.159641,53.910000,158.559998,27.332500,65.889999,90.459999,37.310001,44.900002,NaN,...,92.830002,51.930000,36.231884,38.080002,NaN,52.012939,109.310677,77.430000,28.290001,43.310001
2015-01-05,39.799999,35.948879,53.880001,156.470001,26.562500,64.650002,89.690002,37.070000,44.910000,NaN,...,90.290001,51.570000,35.441368,35.709999,NaN,50.956146,113.388351,76.339996,27.230000,43.049999
2015-01-06,39.180000,36.213211,53.040001,156.360001,26.565001,64.330002,90.180000,36.130001,44.400002,NaN,...,89.809998,50.930000,34.967064,35.500000,NaN,50.330696,112.427185,75.790001,26.190001,42.630001
2015-01-07,39.700001,37.150379,53.009998,159.720001,26.937500,66.930000,91.980003,37.279999,44.759998,NaN,...,90.720001,52.250000,35.415020,35.779999,NaN,51.998562,115.223305,77.720001,26.440001,43.509998
2015-01-08,40.889999,38.207699,53.660000,161.119995,27.972500,67.629997,92.190002,38.959999,45.680000,NaN,...,92.230003,53.540001,36.389988,36.049999,NaN,52.904385,116.446602,79.379997,26.809999,44.180000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-25,119.680000,28.230000,14.580000,64.080002,178.610001,146.690002,179.479996,NaN,104.209999,NaN,...,108.250000,37.529999,15.540000,101.169998,NaN,129.740005,115.239998,268.390015,34.130001,182.820007
2023-08-28,119.919998,28.480000,14.730000,65.629997,180.190002,147.429993,180.389999,NaN,102.790001,NaN,...,109.160004,37.650002,15.910000,101.820000,NaN,129.309998,117.529999,269.920013,34.900002,186.979996
2023-08-29,122.019997,29.610001,14.900000,66.940002,184.119995,147.589996,179.979996,NaN,103.870003,NaN,...,109.809998,37.790001,16.000000,102.290001,NaN,128.899994,119.230003,273.489990,35.439999,192.770004


In [23]:
def pricefilter_rem(ticker):
    df[ticker] = df[ticker][df[ticker].index <= removed[removed.Ticker == ticker].index[0]]

In [24]:
def pricefilter_add(ticker):
    df[ticker] = df[ticker][df[ticker].index >= 
                            overall[overall.Symbol == ticker]['Date added'].values[0]]

In [25]:
for ticker_rem in removed.Ticker:
    pricefilter_rem(ticker_rem)
    
for ticker_add in overall.Symbol:
    pricefilter_add(ticker_add)

In [26]:
# df

In [27]:
ret_df = df.pct_change()

In [28]:
ret_df

,A,AA,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACE,...,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,-0.018738,-0.057935,NaN,-0.013181,-0.028172,-0.018819,-0.008512,-0.006433,0.000223,NaN,...,-0.027362,-0.006932,-0.021818,-0.062237,NaN,-0.020318,0.037304,NaN,-0.037469,-0.006003
2015-01-06,-0.015578,0.007353,NaN,-0.000703,0.000094,-0.004950,0.005463,-0.025357,-0.011356,NaN,...,-0.005316,-0.012410,-0.013383,-0.005881,NaN,-0.012274,-0.008477,NaN,-0.038193,-0.009756
2015-01-07,0.013272,0.025879,NaN,0.021489,0.014022,0.040417,0.019960,0.031829,0.008108,NaN,...,0.010133,0.025918,0.012811,0.007887,NaN,0.033138,0.024870,NaN,0.009546,0.020643
2015-01-08,0.029975,0.028461,NaN,0.008765,0.038422,0.010459,0.002283,0.045064,0.020554,NaN,...,0.016645,0.024689,0.027530,0.007546,NaN,0.017420,0.010617,NaN,0.013994,0.015399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-25,0.001590,0.000000,0.000686,-0.056398,0.012643,0.000409,0.003972,0.000000,0.004434,NaN,...,0.017866,0.007787,0.000000,0.015253,NaN,0.001544,0.008048,0.009289,-0.009864,0.006940
2023-08-28,0.002005,0.000000,0.010288,0.000000,0.008846,0.005045,0.005070,0.000000,-0.013626,NaN,...,0.008407,0.003198,0.000000,0.006425,NaN,-0.003314,0.019872,0.005701,0.022561,0.022755
2023-08-29,0.017512,0.000000,0.011541,0.000000,0.021810,0.001085,-0.002273,0.000000,0.010507,NaN,...,0.005955,0.003718,0.000000,0.004616,NaN,-0.003171,0.014464,0.013226,0.015473,0.030966


In [29]:
# monthly returns:
mtl_ret = (ret_df + 1).resample('M').prod()

In [30]:
mtl_ret

,A,AA,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACE,...,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-31,0.931213,0.985516,1.000000,1.002775,1.071618,0.915920,1.050741,1.386760,0.996882,1.0,...,0.941721,0.963412,0.957818,0.895483,1.0,0.999032,0.995648,1.000000,0.846942,0.986608
2015-02-28,1.117554,0.945048,1.000000,0.974402,1.096449,1.002486,1.081115,1.174913,1.058311,1.0,...,1.012812,1.059564,1.036447,1.046921,1.0,1.122164,1.073952,1.000000,1.116027,1.078633
2015-03-31,0.984364,0.873563,0.970221,0.966178,0.968628,0.967603,1.106170,1.177496,0.978045,1.0,...,0.960018,0.960008,0.941392,0.980952,1.0,0.970534,0.976161,1.000000,1.009723,1.004339
2015-04-30,0.995668,1.038700,0.914930,0.955308,1.005786,1.104544,1.005542,0.883208,1.001943,1.0,...,1.027882,1.002162,0.894942,1.057110,1.0,1.091972,0.934649,1.000000,1.049630,0.959602
2015-05-31,0.995649,0.931446,0.877407,1.071469,1.040991,1.029848,0.984777,0.944638,1.046963,1.0,...,0.975163,1.020392,0.993044,0.987844,1.0,1.048278,1.038693,1.000000,1.019054,1.120441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30,0.978965,1.000000,0.924746,1.032234,1.028987,0.948234,1.042096,1.000000,1.090954,1.0,...,1.079154,1.067464,1.000000,0.991786,1.0,1.064355,1.071517,0.905755,0.930839,1.056116
2023-05-31,0.854094,1.000000,1.083578,0.580658,1.044613,0.912917,1.019778,1.000000,0.923328,1.0,...,0.863444,0.861436,1.000000,0.964946,1.0,0.915422,0.919821,0.911606,0.979541,0.927352
2023-06-30,1.039595,1.000000,1.213802,0.964467,1.094330,0.976587,1.130943,1.000000,1.068823,1.0,...,1.049618,1.107973,1.000000,1.123952,1.0,1.076618,1.143396,1.126671,0.984243,1.056438


In [32]:
# rolling average:
mtl_12 = mtl_ret.rolling(12).apply(np.prod).dropna()

In [33]:
mtl_12

,A,AA,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACE,...,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2015-12-31,1.030819,0.621536,0.778493,0.949231,0.962773,0.899074,1.146474,2.419726,1.000223,1.0,...,0.839707,1.171770,0.773091,0.958508,1.0,1.009675,0.911182,1.000000,0.965005,1.106442
2016-01-31,0.996823,0.465815,0.716728,0.956289,0.830830,0.909694,0.942241,1.649208,0.845621,1.0,...,0.890528,1.177094,0.740319,1.054252,1.0,1.001245,0.885459,1.000000,0.946578,1.007489
2016-02-29,0.884861,0.603786,0.753676,0.958110,0.752686,0.902645,0.842935,1.316170,0.817817,1.0,...,0.905241,1.149972,0.704029,1.047899,1.0,0.893478,0.804137,1.000000,0.797307,0.890866
2016-03-31,0.959085,0.741486,0.776999,1.071147,0.875914,0.975743,0.761415,1.324532,0.902871,1.0,...,0.983412,1.211043,0.868482,1.167904,1.0,1.039761,0.907335,1.000000,0.896667,0.957658
2016-04-30,0.989123,0.832340,0.718368,1.091608,0.749021,0.943396,0.744532,1.536539,0.838001,1.0,...,1.011789,1.168627,0.834783,1.128579,1.0,0.925547,1.053988,1.000000,0.971066,1.058757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30,1.135491,1.000000,0.726692,0.628813,1.076308,1.028867,1.102849,1.329542,0.973304,1.0,...,1.388152,1.048512,1.000000,1.289938,1.0,1.201436,1.146501,0.779175,0.493010,0.991707
2023-05-31,0.906789,1.000000,0.827084,0.383914,1.190876,0.936147,1.099231,1.444899,0.868381,1.0,...,1.064375,0.913043,1.000000,1.189317,1.0,1.059439,1.059313,0.776398,0.478436,0.953665
2023-06-30,1.012461,1.000000,1.414827,0.406147,1.418739,0.879668,1.360122,1.539413,1.003405,1.0,...,1.252335,1.120067,1.000000,1.440522,1.0,1.220597,1.385875,1.006396,0.527701,1.001862
